In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import _model as model

### Load training and testing dataset

In [2]:
train_dataset = keras.utils.image_dataset_from_directory(
    "./fish_train_data",
    label_mode="int",
    color_mode="rgb",
    batch_size=64,
    image_size=(512, 512),
    shuffle=True
)
test_dataset = keras.utils.image_dataset_from_directory(
    "./fish_test_data",
    label_mode="int",
    color_mode="rgb",
    batch_size=64,
    image_size=(512, 512),
    shuffle=True
)

Found 1481 files belonging to 20 classes.
Found 361 files belonging to 20 classes.


### Display one image from the first batch

In [ ]:
for inputs in train_dataset.take(1):
    imgs, labels = inputs

plt.imshow(tf.cast(imgs[2], tf.dtypes.int32))

### Create augmentation layer

In [3]:
augmentation_layers = model.make_aug(fill_value=127.0)

### Show the same image from earlier, but augmented

In [ ]:
plt.imshow(
    tf.cast(augmentation_layers(tf.expand_dims(imgs[2], 0), training=True)[0], tf.dtypes.int32)
)

### Training only the top dense layers

In [4]:
model1 = model.make_model(n_classes=20)
efficient_net = model1.layers[1]
efficient_net.Trainable = False
lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(
    0.03, 1000
)
model1.compile(
    optimizer=keras.optimizers.SGD(learning_rate=lr_decayed_fn, momentum=0.9),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
history = model1.fit(
    train_dataset,
    epochs=50,
    validation_data=test_dataset,
    validation_freq=5
)

Epoch 1/50
24/24 [==============================] - 18s 315ms/step - loss: 2.7405 - accuracy: 0.1722
Epoch 2/50
24/24 [==============================] - 8s 289ms/step - loss: 1.6353 - accuracy: 0.5469
Epoch 3/50
24/24 [==============================] - 8s 285ms/step - loss: 1.0245 - accuracy: 0.6806
Epoch 4/50
24/24 [==============================] - 8s 288ms/step - loss: 0.9826 - accuracy: 0.6860
Epoch 5/50
24/24 [==============================] - 10s 378ms/step - loss: 0.8470 - accuracy: 0.7265 - val_loss: 0.6174 - val_accuracy: 0.8061
Epoch 6/50
24/24 [==============================] - 8s 288ms/step - loss: 0.9776 - accuracy: 0.6921
Epoch 7/50
24/24 [==============================] - 8s 286ms/step - loss: 0.8518 - accuracy: 0.7353
Epoch 8/50
24/24 [==============================] - 8s 289ms/step - loss: 0.8458 - accuracy: 0.7400
Epoch 9/50
24/24 [==============================] - 8s 288ms/step - loss: 0.6898 - accuracy: 0.7927
Epoch 10/50
24/24 [==============================] - 9s 

### Store top model weights

In [13]:
model1.layers[-1].save_weights("trained_weights/trained_top")
del model1

### Load saved model

In [5]:
model2 = model.make_model(n_classes=20)
model2.layers[-1].load_weights("trained_weights/trained_top")

### Run model on locally saved image

In [12]:
img = keras.utils.load_img("salmon.jpg", target_size=(512, 512))
batch = np.expand_dims(keras.utils.img_to_array(img), axis=0)
model2.predict(batch)

1/1 [==============================] - 3s 3s/step


array([[0.0315797 , 0.01744416, 0.04716867, 0.01206983, 0.20707177,
        0.01890964, 0.00788425, 0.00315444, 0.00250336, 0.15584128,
        0.1785851 , 0.01913932, 0.00060353, 0.07811665, 0.01061875,
        0.00892797, 0.00837946, 0.0144966 , 0.15275924, 0.02474626]],
      dtype=float32)